In [1]:
import os
import mujoco
import math
import numpy
import mediapy as media
import matplotlib.pyplot as plt

# import distutils.util
# import subprocess

In [2]:
width = 640
height = 480

In [3]:
xml_template = """
<mujoco>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .15" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 .1">
        <joint type="free"/>
        <geom type="box" size=".05 .05 .003" pos="0 0 0" rgba="1 0 0 1"/>
        
      <body name="leg1_l1" pos=".03 -.05 0">
          <joint name="leg1_j1" type="hinge" axis="1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .02 .001" pos="0 -.02 0" rgba="1 0 0 1"/>
          <body name="leg1_l2" pos=".02 0 0">
            <joint name="leg1_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos=".01 -.02 0" rgba="1 0 0 1"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 -.03 0" rgba="1 0 0 1"/>
            <body name="leg1_l4" pos=".02 0 0">
              <joint name="leg1_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos=".01 -.02 0" rgba="1 0 0 1"/>
            </body>
          </body>
          <body name="leg1_l3" pos="-.02 0 0">
            <joint name="leg1_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos="-.01 -.02 0" rgba="1 0 0 1"/>
            <body name="leg1_l4_2" pos="-.02 0 0">
              <joint name="leg1_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos="-.01 -.02 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

      <body name="leg2_l1" pos=".03 .05 0">
          <joint name="leg2_j1" type="hinge" axis="-1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .02 .001" pos="0 .02 0" rgba="1 0 0 1"/>
          <body name="leg2_l2" pos=".02 0 0">
            <joint name="leg2_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos=".01 .02 0" rgba="1 0 0 1"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 .03 0" rgba="1 0 0 1"/>
            <body name="leg2_l4" pos=".02 0 0">
              <joint name="leg2_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos=".01 .02 0" rgba="1 0 0 1"/>
            </body>
          </body>
          <body name="leg2_l3" pos="-.02 0 0">
            <joint name="leg2_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos="-.01 .02 0" rgba="1 0 0 1"/>
            <body name="leg2_l4_2" pos="-.02 0 0">
              <joint name="leg2_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos="-.01 .02 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

    </body>
  </worldbody>

  
  <!--
   <contact>
    <exclude body1="leg1_l1" body2="leg1_l2" />
    <exclude body1="leg1_l1" body2="leg1_l3" />
    <exclude body1="leg1_l1" body2="leg1_l4" />
    <exclude body1="leg1_l1" body2="leg1_f1" />
    <exclude body1="leg1_l1" body2="leg1_f2" />
    <exclude body1="leg1_l2" body2="leg1_l3" />
    <exclude body1="leg1_l2" body2="leg1_l4" />
    <exclude body1="leg1_l2" body2="leg1_f1" />
    <exclude body1="leg1_l2" body2="leg1_f2" />
    <exclude body1="leg1_l3" body2="leg1_l4" />
    <exclude body1="leg1_l3" body2="leg1_f1" />
    <exclude body1="leg1_l3" body2="leg1_f2" />
    <exclude body1="leg1_l4" body2="leg1_f1" />
    <exclude body1="leg1_l4" body2="leg1_f2" />
    <exclude body1="leg1_f1" body2="leg1_f2" />
  </contact>

-->
  <actuator>
    <position name="m1" joint="leg1_j1"/>
    <position name="m2" joint="leg1_j2"/>
    <position name="m3" joint="leg2_j1"/>
    <position name="m4" joint="leg2_j2"/>
  </actuator>
  
  <equality>
    <weld name="weld1" active="true" body1="leg1_l4" body2="leg1_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
    <weld name="weld2" active="true" body1="leg2_l4" body2="leg2_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
  </equality>  
  <!--
  -->
</mujoco>
"""

xml = xml_template.format(width=width,height=height,k=1e-3,b=1e-4)

In [4]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)


duration = 10  # (seconds)
framerate = 30  # (Hz)


A00 = 30 * math.pi/180
A11 = 30 * math.pi/180
f = 1
t0_1 = 0
t0_2 = .25
t0_3 = .5
t0_4 = .75
b1 = -30*math.pi/180
b2 = -90*math.pi/180
b3 = -30*math.pi/180
b4 = -90*math.pi/180

def my_controller(model, data):
    t = data.time
    if t<1:
      A0=0
      A1=0
    elif t<2:
      A0 = A00*(t-1)/(2-1)
      A1 = A11*(t-1)/(2-1)
    else:
      A0 = A00
      A1 = A11
    j1 = A0*math.sin(2*math.pi*(f*t-t0_1))+b1
    j2 = A1*math.sin(2*math.pi*(f*t-t0_2))+b2
    j3 = A0*math.sin(2*math.pi*(f*t-t0_3))+b3
    j4 = A1*math.sin(2*math.pi*(f*t-t0_4))+b4
    data.ctrl = [j1,j2,j3,j4]
    return

mujoco.set_mjcb_control(my_controller)

frames = []
mujoco.mj_resetData(model, data)
data.qpos[0:3] = numpy.array([0,0,.25])
data.qpos[7:12] = numpy.array([0,-90,-90,90,90])* math.pi/180
data.qpos[12:17] = numpy.array([0,-90,-90,90,90])* math.pi/180
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data,camera='target')
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate,width=width,height=height)

This browser does not support the video tag.


In [5]:
data.qpos

array([ 0.12602431,  0.01140059,  0.01198299,  0.99696399,  0.01635841,
       -0.0757746 ,  0.00730892, -0.52529298, -2.09786046, -1.03627365,
        1.04395657,  2.09073508, -0.52904798, -1.03751508, -2.09663693,
        2.10338457,  1.03139157])

In [6]:
data.tendon

<bound method PyCapsule.tendon of <mujoco._structs.MjData object at 0x7f14a28dd170>>